# Pre-Workshop Preparation: Getting Started with Google Earth Engine and Python in Google Colab

To get the most out of the upcoming workshop, please complete the following setup steps ahead of time. This will ensure you're ready to dive into hands-on exercises using Google Earth Engine (GEE) and Python via Google Colab.

1. Sign Up for Google Earth Engine
Google Earth Engine is a cloud-based platform for large-scale geospatial analysis. To use it:

* Visit the Google Earth Engine sign-up page: https://earthengine.google.com/
**Sign in with your Google account and request access (approval usually takes 1–2 days)**

  *Once approved, you can explore the Code Editor at code.earthengine.google.com*

2. Familiarize Yourself with the Earth Engine Code Editor
While we’ll be using Python in the workshop, the Code Editor (which uses JavaScript) is a great way to explore datasets and understand GEE’s structure. Take a few minutes to explore:

* The Script Panel (for writing code)
* The Map Panel (for visualizing results)
* The Inspector Tool (to examine pixel values)
* The Docs tab (for dataset and function references)

3. Set Up Google Colab for Earth Engine
We’ll be using Google Colab, a free, cloud-based Jupyter notebook environment that runs in your browser—no installation required.

**Before the workshop:**

Open Google Colab
Run the following code in a new notebook to install and authenticate Earth Engine:


In [ ]:

!pip install earthengine-api geemap
import ee
ee.Authenticate()
ee.Initialize()



⚠️ You’ll be prompted to log in with your Google account and paste an authentication code—this step is required only once per session.

4. Optional: Explore geemap
geemap is a Python package that integrates Earth Engine with interactive maps in Colab. It simplifies visualization and spatial analysis.

In [ ]:
# EXAMPLE - Visualize trends in precipitation In algonquin over the last 10 years

# Install required packages
!pip install earthengine-api geemap

import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Define the region of interest: Algonquin Provincial Park (approximate bounding box)
algonquin = ee.Geometry.Rectangle([-79.0, 45.2, -77.5, 46.2])

# Define the time range
start_date = '2015-01-01'
end_date = '2025-01-01'

# Load ERA5 monthly precipitation data
era5 = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY') \
    .filterDate(start_date, end_date) \
    .select('total_precipitation')

# Function to calculate mean precipitation over the region
def extract_precipitation(image):
    date = image.date().format('YYYY-MM')
    mean_dict = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=algonquin,
        scale=10000,
        maxPixels=1e9
    )
    return ee.Feature(None, {
        'date': date,
        'precip_mm': ee.Number(mean_dict.get('total_precipitation')).multiply(1000)  # Convert from meters to mm
    })

# Apply the function to each image
precipitation_features = era5.map(extract_precipitation).filter(ee.Filter.notNull(['precip_mm']))

# Extract data to Python
dates = precipitation_features.aggregate_array('date').getInfo()
values = precipitation_features.aggregate_array('precip_mm').getInfo()

# Create a DataFrame
df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'Precipitation (mm)': values
}).sort_values('Date')

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Precipitation (mm)'], marker='o', linestyle='-')
plt.title('Monthly Precipitation in Algonquin Provincial Park (2015–2025)')
plt.xlabel('Date')
plt.ylabel('Precipitation (mm)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
